# 3D printing pluto dust

In [ ]:
from pathlib import Path
import sys
import imageio

import numpy as np

from scipy.interpolate import RegularGridInterpolator

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from IPython.display import display

import astro3d
from astro3d.image_stack import makeslice, process

plt.style.use([{'image.cmap':'gray_r'}])

Read data.

In [ ]:
with np.load('../../../data/pluto_data_norm.npz') as f:
    data = f['rho']
    x = f['x']
    y = f['y']
    z = f['z']

## Normalization

Find the largest magnitude of the data values and define a logarithmic norm

In [ ]:
dyn_range = 1e-4
vmax = 10**np.ceil(np.log10(data.max()))
norm = LogNorm(dyn_range * vmax, vmax, clip=True)

## Example plot

Select which slice to plot

In [ ]:
# Select which slice to plot
i = int(np.ceil(data.shape[-1] / 2)) # midplane or just above if nz is even

# apply the norm
d_0 = np.array(norm(data[:, :, i]))

# plot it and it's dithered version
f, ax = plt.subplots(1, 2, figsize=(10, 5), dpi=150)
ax[0].imshow(d_0, vmin=0, vmax=1)
ax[1].imshow(astro3d.fmodule.dither(d_0), vmin=0, vmax=1);

## Upscale the data

### Coordinates & aspect ratios

create an interpolation function for the non-normalized 3D data

In [ ]:
f_interp = RegularGridInterpolator((x, y, z), data)

settings of the printer (printer specific and layer thickness can be chosen to be different)

In [ ]:
# these are the values for the J850 Prime
#dpi_x = 600
#dpi_y = 600
#dpi_z = 940  # 0.027 mm layer thickness

# these are the values used in alphacams TEILEFABRIK, where we ordered some of our prints
dpi_x = 600
dpi_y = 300
dpi_z = 940 # 0.027 mm layer thickness = 2.54 / dpi_z

Choose the height of the print, the rest should rescale accordingly

In [ ]:
height = 2 # this should be the total height of the printed cube in cm

calculate the new grids in x, y, z

In [ ]:
#n_z = int(height / layer_thickness)
n_z = int(height * dpi_z / 2.54)

n_x = int(n_z * len(x) / len(z) / dpi_z * dpi_x)
n_y = int(n_z * len(y) / len(z) / dpi_z * dpi_y)

n_x += n_x%2 # add 1 to make it even if it isn't
n_y += n_y%2 # add 1 to make it even if it isn't

x2 = np.linspace(x[0], x[-1], n_x)
y2 = np.linspace(y[0], y[-1], n_y)
z2 = np.linspace(z[0], z[-1], n_z)

define the colors for color prints

In [ ]:
colors = np.array([
    [255, 255, 255],
    [255, 0.0, 0.0],
    [0.0, 255, 0.0],
    [0.0, 0.0, 255],
])

# the indices of the colors
lst = np.arange(len(colors))

the CMY colors of the printer

In [ ]:
colors_CMY = np.array([
    [0.0, 93, 127],
    [166, 33, 98],
    [200, 189, 17],
])

# the indices of the colors
lst_CMY = np.arange(len(colors_CMY))

### Iteration

we get the new layer by interpolating the 3D data. We store the images in the path set by `output_dir`.

In [ ]:
output_dir = 'slices_pluto_color2'

Prepare output folder

In [ ]:
path = Path(output_dir)

if not path.is_dir():
    path.mkdir()
else:
    files = list(path.glob('slice*.png'))
    if len(files)>0:
        print('directory exists, deleting old files')
        for file in files:
            file.unlink()

select which index in the new z-grid to process

In [ ]:
iz = int(np.ceil(n_z / 2))

This cell does the same as `makeslice`: interpolates one layer, creates and dithers the image and writes it to file

In [ ]:
# update coordinates - only last entry changes
_x, _y, _z = np.meshgrid(x2, y2, [z2[0]], sparse=False, indexing='ij')
_z = np.array([[[z2[iz]]]])
coords = (_x, _y, _z)

# interpolate: note that we transpose here as this is how the image will be saved
new_layer = f_interp(coords)[:, :, 0].T

# normalize, convert to grayscale image
layer_norm = np.array(norm(new_layer))
layer_dither = astro3d.fmodule.dither(layer_norm)

# save as png
imageio.imwrite(path / f'slice_{iz:04d}.png', np.uint8(255 - 255 * layer_dither))

In [ ]:
f, axs = plt.subplots(3, 1, dpi=100, figsize=(2*3, 3*3), constrained_layout=True)
axs[0].imshow(norm(data[:,:,round(iz / n_z * data.shape[-1])]).T, vmin=0, vmax=1, origin='lower')
axs[1].imshow(layer_norm, vmin=0, vmax=1, origin='lower')
axs[2].imshow(layer_dither, vmin=0, vmax=1, origin='lower')
axs[0].text(0.05, 0.95, 'step 1: original data, log-normalized', fontsize='small', transform=axs[0].transAxes)
axs[1].text(0.05, 0.95, 'step 2: interpolated to printer dimension', fontsize='small', transform=axs[1].transAxes)
axs[2].text(0.05, 0.95, 'step 3: dithered', fontsize='small', transform=axs[2].transAxes)

for ax in axs:
    ax.set_ylabel('y [pixel]')
    ax.set_anchor('W')
axs[-1].set_xlabel('x [pixel]');

# color prints

In [ ]:
_x, _y, _z = np.meshgrid(x2, y2, [z2[0]], sparse=False, indexing='ij')
_z = np.array([[[z2[iz]]]])
coords = (_x, _y, _z)


for iz in [n_z//2]:# range(739):
    
    coords = (_x, _y, np.array([[[z2[iz]]]]))

    # interpolate
    new_layer = f_interp(coords)[:, :, 0].T

    # normalize, convert to grayscale image
    layer_norm = np.array(norm(new_layer))
    layer_dither = astro3d.fmodule.dither(layer_norm)

    #convert to RGBA scale and save as a numpy array
    a = np.array(255 - 255 * np.ones([*layer_dither.shape, 3]) * layer_dither[:, :, None], dtype=np.uint8)

    # imageio.imwrite(path / f'slice_{iz:04d}.png', a, optimize=True, bits=32)

these are the thresholds used in the rendering: `[0.15, 0.42, 0.7]`. Here we compute what density values they correspond to on our scale. We also assign a width to each density contour and a filling factor.

In [ ]:
rho_i = norm.inverse([0.15, 0.42, 0.7]).data

sig = [0.05, 0.03, 0.04]
fill = [0.1, 0.1, 0.1]

Show a histogram of the data values

In [ ]:
bins = np.geomspace(dyn_range * vmax, vmax, 100)
counts, _ = np.histogram(data.ravel(), bins=bins)

In [ ]:
f, ax = plt.subplots()
ax.bar(bins[:-1], counts, align='edge', width=np.diff(bins))
ax.set_xscale('log')
ax.set_yscale('log')
ax2 = ax.secondary_xaxis('top', functions=(norm, norm.inverse))
ax2.set_xscale('linear')
for _rho, _sig in zip(rho_i, sig):
    ax.axvline(_rho, c='k', ls='--')
    ax.errorbar(_rho, 10.**np.mean(np.log10(ax.get_ylim())),
                xerr=[
                    [_rho - norm.inverse(norm(_rho) - _sig)],
                    [norm.inverse(norm(_rho) + _sig) - _rho]], c='k', capsize=5)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
#create mask
dist_sq = (np.log10(np.array(rho_i)[None, None, :] / new_layer[..., None]) / sig)**2
mask = (np.random.rand(*dist_sq.shape)<=fill) * np.random.randn(*dist_sq.shape)**2 > dist_sq


# assign probabilities for each color
p = np.concatenate(((mask.sum(-1)==0)[:, :, None], mask), axis=-1)
p = p/p.sum(-1)[:, :, None]


# loop over all pixels
for i in tqdm(range(mask.shape[0])):
    for j in range(mask.shape[1]):

        # randomly pick one of the colors where mask is True (using the probabilities `p`)
        idx = np.random.choice(lst, p=p[i,j])

        # assign that color to the pixel
        a[i,j,:] = colors[idx, :]

In [ ]:
%%file fort.f90

subroutine color_dither()
#create mask
dist_sq = (np.log10(np.array(rho_i)[None, None, :] / new_layer[..., None]) / sig)**2
mask = (np.random.rand(*dist_sq.shape)<=fill) * np.random.randn(*dist_sq.shape)**2 > dist_sq


# assign probabilities for each color
p = np.concatenate(((mask.sum(-1)==0)[:, :, None], mask), axis=-1)
p = p/p.sum(-1)[:, :, None]


# loop over all pixels
for i in tqdm(range(mask.shape[0])):
    for j in range(mask.shape[1]):

        # randomly pick one of the colors where mask is True (using the probabilities `p`)
        idx = np.random.choice(lst, p=p[i,j])

        # assign that color to the pixel
        a[i,j,:] = colors[idx, :]


In [ ]:
    
    #convert red pixels into 75% magenta 25% yellow, green pixels into 50% cyan 50% yellow, blue pixels into 60% cyan 40% magenta
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            idx_r = np.random.choice(lst_CMY, p=[0, 0.75, 0.25])
            idx_g = np.random.choice(lst_CMY, p=[0.5, 0, 0.5])
            idx_b = np.random.choice(lst_CMY, p=[0.6, 0.4, 0])
            if all(a[i,j] == [255, 0.0, 0.0]):
                a[i,j] = colors_CMY[idx_r, :]
            elif all(a[i,j] == [0.0, 255, 0.0]):
                a[i,j] = colors_CMY[idx_g, :]
            elif all(a[i,j] == [0.0, 0.0, 255]):
                a[i,j] = colors_CMY[idx_b, :]
    
    #convert img array back into image
    im = Image.fromarray(a, mode="RGB")
    
    # save as 1bit bitmap
    im.save(path / f'colored_slice_CMY_{iz:04d}.png', bits=1, optimize=True)

this is the same result using `makeslice`, but one can select between 1 or 32 bit. Fit.technology wants 32 bit.

In [ ]:
makeslice(iz, z2, f_interp, coords, norm, path, bits=32, fg=[255, 255, 255, 255], bg=[1, 1, 1, 250])

get the colors in that slice

In [ ]:
im = Image.open("slices_fit2/slice_0000.png")
colors = np.unique(np.array(im).reshape(-1, 4), axis=0)
display(colors)
display(im)

## Batch processing

all of the above can also be done in a loop with `process`:
normalizing with the given norm, up-scaling and saving to images. We'll just do this same one here by specifying the `iz` keyword.

Here we just want to print the first cm.

In [ ]:
iz = np.arange(int(dpi_z/2.54))

In [ ]:
process(data,
        height=height, dpi_x=dpi_x, dpi_y=dpi_y, dpi_z=dpi_z,
        output_dir=output_dir,
        norm=norm,
        fg=[255, 255, 255, 255], bg=[1, 1, 1, 250]
        #iz=iz
       )